In [1]:
# Import libraries
import numpy as np
import geopandas as gpd
import momepy
import networkx as nx
# import pandas as pd
# import shapely
# import shapely.geometry as sg
# import matplotlib
# import matplotlib.pyplot as plt
# %matplotlib inline

from lmzintgraf_gp_pref_elicit import dataset, gaussian_process, acquisition_function
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_ccs as utils_ccs
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_data as utils_data
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_experiment as utils_experiment
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_parameters as utils_parameters
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_user as utils_user

In [2]:
map = gpd.read_file("Sidewalk_width_crossings_small.geojson") #Read in the map with radius 250m and ~1000 nodes

# Objectives
objective1 = map['length']
objective2 = map['crossing']
objective3 = map['obstacle_free_width']

objectives = ('length', 'crossings')

In [3]:
# Create a NetworkX graph from the map
G = momepy.gdf_to_nx(map, approach='primal')
nodes = G.nodes
edges = G.edges

/usr/local/anaconda3/envs/test/lib/python3.10/site-packages/momepy/utils.py:252: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_network[length] = gdf_network.geometry.length


In [4]:
# print(nodes)

In [5]:
#Pick random ones or pick manually that make sense - to experiment
S = (120548.6120283842, 486088.19577846595)
T = (120798.0893320718, 486128.7633437495)

In [6]:
# Initialise the Gaussian process for 2 objectives
gp = gaussian_process.GPPairwise(num_objectives=2, std_noise=0.01, kernel_width=0.15,prior_mean_type='zero', seed=None)

In [41]:
P = [] #Pareto set
p = []

# Path initialisation
for i in objectives:
    p = nx.shortest_path(G, source=S, target=T, weight=i, method='dijkstra') #Dijkstra algorithm
    P.append(p)

[[(120548.6120283842, 486088.19577846595),
  (120558.58272730978, 486088.5913559315),
  (120562.22249291666, 486088.4604878346),
  (120597.87366172913, 486118.3885286597),
  (120603.94807229919, 486112.8244804511),
  (120610.1211584085, 486110.4734641692),
  (120615.236697444, 486109.4981894336),
  (120622.18953041689, 486108.8516084536),
  (120627.6055477486, 486108.80491815176),
  (120637.3711853829, 486107.4803808584),
  (120638.32762784461, 486107.6578267999),
  (120644.82334925648, 486107.55756407854),
  (120646.58367929762, 486107.4539825946),
  (120653.63177241654, 486106.9806947992),
  (120658.16234661153, 486106.63926980837),
  (120664.93105615664, 486106.10542439454),
  (120671.8192466126, 486105.5397502568),
  (120678.46251254133, 486104.92881432245),
  (120683.05468505199, 486104.3662392544),
  (120689.70658339545, 486103.4113373837),
  (120697.63022968677, 486102.02209520293),
  (120704.74846380341, 486100.38072378177),
  (120714.24315622302, 486097.30355140124),
  (120715

In [42]:
C = min(P, key=lambda p: (p[0], p[1]))
C

[(120548.6120283842, 486088.19577846595),
 (120558.58272730978, 486088.5913559315),
 (120562.22249291666, 486088.4604878346),
 (120597.87366172913, 486118.3885286597),
 (120603.94807229919, 486112.8244804511),
 (120610.1211584085, 486110.4734641692),
 (120615.236697444, 486109.4981894336),
 (120622.18953041689, 486108.8516084536),
 (120627.6055477486, 486108.80491815176),
 (120637.3711853829, 486107.4803808584),
 (120638.32762784461, 486107.6578267999),
 (120644.82334925648, 486107.55756407854),
 (120646.58367929762, 486107.4539825946),
 (120653.63177241654, 486106.9806947992),
 (120658.16234661153, 486106.63926980837),
 (120664.93105615664, 486106.10542439454),
 (120671.8192466126, 486105.5397502568),
 (120678.46251254133, 486104.92881432245),
 (120683.05468505199, 486104.3662392544),
 (120689.70658339545, 486103.4113373837),
 (120697.63022968677, 486102.02209520293),
 (120704.74846380341, 486100.38072378177),
 (120714.24315622302, 486097.30355140124),
 (120715.2451666701, 486096.9044

In [43]:
# User ranking: Compare paths in P
user_preference = utils_user.UserPreference(num_objectives=2, std_noise=0.1)

In [10]:
#TODO: This compiles for eternity. Idk why :'(
# user_preference.rescale_on_input_domain(C)

In [11]:
#TODO: This gives an error from x=x[np.argsort(-utility)] from utils_user.py. So I used user_preference.get_preference (aka the next jupyter code cell, but idk if it's the correct method.
#user_ranking = user_preference.ranking(C, add_noise=True)

/Users/alisatodorova/Desktop/Thesis/Interactive-MOPPA/lmzintgraf_gp_pref_elicit/gp_utilities/utils_user.py:254: RuntimeWarning: overflow encountered in exp
  y += 1. / (1 + np.exp(- x * (a-i) + (b+i)))


TypeError: only integer scalar arrays can be converted to a scalar index

In [44]:
add_noise = True
preference = user_preference.get_preference(C,add_noise=add_noise)
print(preference)

[530.42805579 530.56744835 530.60425433 530.5119096  530.48131299
 530.47843775 530.67360493 530.4653542  530.58678176 530.39384069
 530.52954345 530.52430106 530.56549686 530.46126191 530.5109967
 530.59469654 530.26919015 530.65040723 530.43880573 530.71304689
 530.64361791 530.55674119 530.53648675 530.477934   530.67274628
 530.59666211 530.5583101  530.38861385 530.44760363 530.3695689
 530.63323236 530.5371369  530.63357459 530.56695449 530.68589299
 530.56645352 530.65430298 530.44242188 530.63374862 530.59325817
 530.51215665 530.46594289 530.62955655 530.5169277  530.4367286
 530.74838024 530.65272624 530.40373894 530.48649439 530.63209912
 530.49715685 530.56766137 530.51215311 530.50628267 530.36658322
 530.48682098 530.62126948 530.36740502 530.42033818 530.48252391
 530.66087453 530.6727236  530.40218309 530.54693378 530.34289546
 530.61102083 530.26598664 530.42667613 530.42218828 530.43103244
 530.4581224  530.53097746 530.39086787 530.57137417 530.55032901
 530.41874649

/Users/alisatodorova/Desktop/Thesis/Interactive-MOPPA/lmzintgraf_gp_pref_elicit/gp_utilities/utils_user.py:254: RuntimeWarning: overflow encountered in exp
  y += 1. / (1 + np.exp(- x * (a-i) + (b+i)))


In [45]:
# Add the comparisons to GP
comparisons = dataset.DatasetPairwise(num_objectives=2)
preference_copy = np.copy(preference)
comparisons.add_ranked_preferences(preference_copy)
gp.update(comparisons)

In [ ]:
#TODO: This code is working but I don't know for what to use it...
# params = utils_parameters.get_parameter_dict(query_type='ranking', utility_noise=0.01, num_objectives=2)
# params['num queries'] = 10
# params['seed'] = 123
#
# # Run the experiment
# experiment = utils_experiment.Experiment(params)
#
# result = experiment.run(recalculate=False)
#
#  # -- true utility --
#
# # get the input domain
# input_domain = result[1]
# idx_order = np.argsort(-input_domain[:, 0])
#
# # get the true utility function
# true_utility = result[2]
# print(true_utility)

In [59]:
# Find the path the user likes best and has the maximum a posteriori (MAP) estimate
gp_sample = gp.sample(p)
print(gp_sample)

[ 1.61858706  0.95885938 -0.86239643 -0.38045553 -0.31535667  0.48043804
  0.20564092  0.39743799 -0.08745998  1.07331616 -0.31608633 -0.69307391
 -0.17775457  1.70882657  0.27437394 -0.29028185  0.2755979   0.65079226
 -0.91436006 -0.51078988 -0.27749135 -0.2481468   0.07130219 -0.19570038
 -0.49524677  0.10876156  1.07325105 -1.39299806 -0.59199051 -0.81132709
  0.77507443 -0.07459805 -0.11363512 -0.52187832  0.3940336   0.02337216
  0.29129348  0.43465611  0.46753501  0.11677955  0.42169594 -0.79154613
 -1.58642593 -0.53146049  0.32495487 -2.5443497  -0.3100296  -0.4934927
 -0.34917141  0.05441079 -1.00429795 -0.26020559  0.31852289 -0.01898118
  0.44261525  0.58941091  0.40550119 -1.21967472  0.62438663 -0.53478334
 -1.69009402 -1.90988968 -0.13371532  0.45390474  0.15139734 -0.94542023
  0.92294635  1.20731849  0.67108665  0.27193594  0.6343804   0.81851868
  0.87630562  0.50680585  0.08991298 -0.37239174 -1.04768092 -0.81736635
  0.73783621  0.45936621 -0.06190286]


In [61]:
p_star_index = np.argmax(gp_sample)
p_star_index

13

In [64]:
p_star = p[p_star_index]
p_star

(120628.1338571294, 486024.86469161004)

In [65]:
# Initialise the acquisition function
acq_fun = acquisition_function.DiscreteAcquirer(input_domain=C, query_type='ranking', seed=123, acquisition_type='expected improvement')

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
# while C:
#     t = max(C, key=lambda v: acq_fun(v)) #Pick candidate target with highest value acquisition function

In [ ]:
    #TODO: remove t from C

